In [9]:
#on-policy型モンテカルロ法

import numpy as np
import copy

#0～399のsanumberに対して、状態行動の割り当て
def getStateAction(sanumber):
    
    if sanumber % 2==1:
        hit=True
    else:
        hit=False
        
    if sanumber>=200:
        usableA=True
    else:
        usableA=False
        
    n=sanumber // 2
    n=n % 100
    
    sumCards=(n % 10)+12
    dealerCard=(n // 10)+2
    
    sa=[sumCards, dealerCard, usableA, hit]
    return sa

#状態ナンバー(0～199)
def getStateNumber(sumCards, dealerCard, usableA):
    n=(sumCards-12)+10*(dealerCard-2)
    if usableA:
        n+=100
    return n

def pickCard():
    #1～13の整数
    c=np.random.randint(1,14)
    if c>10:
        c=10
    elif c==1:
        c=11
    return c


def playerAct(sanumber):
    #経由した状態行動対の番号の集合(初回訪問MCに対応)
    #リストを使えば逐一訪問MCになる。
    sanumbers=set([sanumber])
    
    sa=getStateAction(sanumber)
    
    sumCards=sa[0]
    dealerCard=sa[1]
    usableA=sa[2]
    hit=sa[3]

    while(hit):
        c=pickCard()
        sumCards+=c
        if c==11:
            usableA=True
        if sumCards>21 and usableA:
            sumCards-=10
            usableA=False
        if sumCards>21:
            break
            
        snumber=getStateNumber(sumCards, dealerCard, usableA)
        #乱数選択
        r=np.random.rand()
        if r<piH[snumber]:
            hit=True
            sanumber=snumber*2+1
        else:
            hit=False
            sanumber=snumber*2
            
        sanumbers.add(sanumber)
        
    result=[sumCards,dealerCard,sanumbers]
    return result
        
def dealerAct(dealerCard):
    usableA=False
    if dealerCard==11:
        usableA=True
    s=dealerCard

    while(s<17):
        c=pickCard()
        s+=c
        if c==11:
            usableA=True
        if s>21 and usableA:
            s-=10
            usableA=False
            
    return s

def getReturn(psum, dsum):
    if psum>21:
        r=-1
    elif dsum>21:
        r=1
    elif psum>dsum:
        r=1
    elif dsum>psum:
        r=-1
    else:
        r=0
        
    return r


#+++問題となる地点までゲームを進行し、状態番号を取得+++
def prepare():
    #ディーラー
    d=pickCard()
    #プレーヤー
    s=0
    usableA=False
    while s<12:
        c=pickCard()
        s+=c
        if s>21: #11からAを引いた時のみ22→12(この場合、Aの有効無効はそれまでの状態を引き継ぐ)
            s-=10
        elif c==11:
            usableA=True
    
    snumber=getStateNumber(s,d,usableA)
    return snumber

#+++各状態から方策に従い行動を決定+++
def decideAction(snumber, piH):
    sanumber=snumber*2
    r=np.random.rand()
    #乱数選択
    if r<piH[snumber]:
        sanumber+=1
    return sanumber

In [10]:
Q=np.random.rand(200,2)*2-1 #Qの初期化　[-1,1]の乱数
piH=np.random.rand(200) #方策の初期化：piHの確率でHit
n=1000 #試行セット数
e=0.1 #εの値。最適でない行動が選ばれる確率はe/2

returns=[]
for i in range(400):
    ret=[] #appendはリストの方が速い？numPy配列は平均値計算が速いがAppendが遅い
    returns.append(ret)
    
table=np.zeros((n,3))

#エピソード400回を１セットとして、nセットのシミュレーション
for i in range(n):
    
    piBfr=copy.deepcopy(piH)
    QBfr=copy.deepcopy(Q)
        
    for j in range(400):
        #+++初期状態を開始点探査でなくゲームの最初からランダム選択+++
        snum_init=prepare()        
        sanum_init=decideAction(snum_init, piH)
        result=playerAct(sanum_init)
        #result=playerAct(j)

        psum=result[0]
        sanumbers=result[2]
        dsum=0
        if psum<=21:
            dealerCard=result[1]
            dsum=dealerAct(dealerCard)
        ret=getReturn(psum,dsum)

        for sanumber in sanumbers:
            rlist=returns[sanumber]
            rlist.append(ret)
            ave=sum(rlist)/len(rlist)
            s=sanumber // 2
            a=sanumber % 2
            Q[s,a]=ave
            #+++方策更新+++            
            if Q[s,1]>Q[s,0]:#Hitの価値が高い時
                piH[s]=1-e/2
                #piH[s]=1
            else:
                piH[s]=e/2
                #piH[s]=0
    
    difPi=piH-piBfr
    difQ=Q-QBfr
    
    table[i,0]=i
    table[i,1]=np.count_nonzero(difPi)
    table[i,2]=int(np.linalg.norm(difQ)*10000)    

In [11]:
#結果の評価

print(Q)
print(piH)

matrixA=np.zeros((11,11))
matrixN=np.zeros((11,11))
valueA=np.zeros((11,11))
valueN=np.zeros((11,11))

for i in range(10):
    matrixA[i+1,0]=i+12 #プレーヤー
    matrixN[i+1,0]=i+12 #プレーヤー
    valueA[i+1,0]=i+12 #プレーヤー
    valueN[i+1,0]=i+12 #プレーヤー
    matrixA[0,i+1]=i+2 #ディーラー
    matrixN[0,i+1]=i+2 #ディーラー
    valueA[0,i+1]=i+2 #プレーヤー
    valueN[0,i+1]=i+2 #プレーヤー

for i in range(200):
    sa=getStateAction(i*2)
    h=piH[i]
    if h>0.5:
        optAct='H'
        v=Q[i,1]
    else:
        optAct='S'
        v=Q[i,0]
    sa[3]=optAct
    sa.insert(0,i)
    print(sa)
        
    player=i%10
    dealer=(i//10)%10
    usableA=i//100
    if usableA==1:
        matrixA[player+1,dealer+1]=h
        valueA[player+1,dealer+1]=int(v*1000)
    else:
        matrixN[player+1,dealer+1]=h
        valueN[player+1,dealer+1]=int(v*1000)
    
print('With A')
print(matrixA)
print('Without A')
print(matrixN)

print('Value with A')
print(valueA)
print('Value without A')
print(valueN)



[[-0.29816661 -0.30995475]
 [-0.30941704 -0.34482759]
 [-0.3276203  -0.42134831]
 [-0.28521127 -0.52760736]
 [-0.30823364 -0.51724138]
 [-0.1704996  -0.60958904]
 [ 0.15127175 -0.625     ]
 [ 0.37929412 -0.76363636]
 [ 0.63594203 -0.79885057]
 [ 0.89540412 -1.        ]
 [-0.32984293 -0.23576042]
 [-0.23835287 -0.29951691]
 [-0.28238806 -0.45989305]
 [-0.21710741 -0.44720497]
 [-0.24856661 -0.57446809]
 [-0.132116   -0.52317881]
 [ 0.14154705 -0.74814815]
 [ 0.38458188 -0.63963964]
 [ 0.6681465  -0.85416667]
 [ 0.87287174 -1.        ]
 [-0.23156612 -0.31726908]
 [-0.36       -0.27919255]
 [-0.20684387 -0.39534884]
 [-0.23228995 -0.4556213 ]
 [-0.22548028 -0.5625    ]
 [-0.08916274 -0.53691275]
 [ 0.14043393 -0.50757576]
 [ 0.43968386 -0.75925926]
 [ 0.6480733  -0.84878049]
 [ 0.89966555 -1.        ]
 [-0.18410676 -0.26966292]
 [-0.16916215 -0.39893617]
 [-0.15336617 -0.28658537]
 [-0.17034483 -0.4301676 ]
 [-0.17565598 -0.5       ]
 [-0.07191781 -0.48076923]
 [ 0.19878735 -0.5862069 ]
 